In [ ]:
# Basic import and reading dataframe
from google.colab import files
import pandas as pd
import numpy as np
reviews_df_main = pd.read_excel("RC.xlsx")
reviews_df = pd.DataFrame()

FileNotFoundError: ignored

In [ ]:
#extracting parts of dataframe that we need into new dataframe
reviews_df['review'] = reviews_df_main["Review"]
reviews_df['OTA'] = reviews_df_main['OTA']
reviews_df

NameError: ignored

In [ ]:
reviews_df.dropna(inplace= True)

reviews_df["review"] = [x.replace("\n", " ") for x in reviews_df["review"] ]
reviews_df["review"] = [x.replace(".", ". ") for x in reviews_df["review"] ]
reviews_df.info()

In [ ]:
#breaking point

#breaking point to select overall or just the OTA wise data 
#ClearTrip', 'Goibibo', 'MMT', 'Redbus', 'Yatra'
abc = "Yatra"
reviews_df = reviews_df[reviews_df["OTA"] == abc ]
#reviews_df



In [ ]:
# return the wordnet object value corresponding to the POS tag
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
reviews_df["review"] = reviews_df["review"].apply(lambda x : str(x))
reviews_df["review_clean"] = reviews_df['review'].apply(lambda x: clean_text(x))
reviews_df.head()

In [ ]:
# add sentiment anaylsis columns
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews_df["sentiments"] = reviews_df["review"].apply(lambda x: sid.polarity_scores(x))
reviews_df = pd.concat([reviews_df.drop(['sentiments'], axis=1), reviews_df['sentiments'].apply(pd.Series)], axis=1)
reviews_df.head()

In [ ]:
reviews_df["words"] = reviews_df["review_clean"].apply(lambda x: x.split(" "))
reviews_df.head()

In [ ]:
totalwords = []
distinctwords = []
word = []
count = []



for i in reviews_df["words"]:
  for j in i:
    if j not in totalwords:
      distinctwords.append(j)

    totalwords.append(j)
    

print(len(totalwords))
#142438

In [ ]:
#overall wordcloud
for i in range(len(distinctwords)): 
        word.append(distinctwords[i])
        count.append(totalwords.count(distinctwords[i]))

#print("{0} : {1}".format(word[0],count[0]) )

wordcount = pd.DataFrame()

wordcount['words'] = pd.Series(word).values
wordcount['count'] = pd.Series(count).values
print(wordcount.head(10))



d1 = {}
for a, x in wordcount.values:
    d1[a] = x

#overall
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud( background_color ='white')
wordcloud.generate_from_frequencies(frequencies=d1)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig(abc+'_overall.png', dpi=600) 

plt.show()

'''
#OTA WISE
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

# Read the whole text.
#text = open(path.join(d, 'alice.txt')).read()
#text = ' '.join(wordcount['words'])
alice_mask = np.array(Image.open(path.join(d, "goibibo.png")))

stopwords = set(STOPWORDS)

wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,
               stopwords=stopwords, contour_width=3, contour_color='steelblue')

# generate word cloud
#wc.generate(text)
wc.generate_from_frequencies(frequencies=d1)

# store to file
wc.to_file(path.join(d, "goibibo_cloud.png"))

# show
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
#plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis("off")
plt.show()'''



In [ ]:
#positive words wordcloud


posWordCount = pd.DataFrame()

positiveWords = []
positiveCount = []
for word in distinctwords:
    if (sid.polarity_scores(word)['compound']) >= 0.5:
        positiveWords.append(word)
        positiveCount.append(totalwords.count(word))

posWordCount['words'] = pd.Series(positiveWords).values
posWordCount['count'] = pd.Series(positiveCount).values
#print(posWordCount.head(10))

d = {}
for a, x in posWordCount.values:
    d[a] = x

import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud( background_color ='white')
wordcloud.generate_from_frequencies(frequencies=d)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig(abc+'_positive.png', dpi=600) 

plt.show()

#print("{0} : {1}".format(positiveWords[1],positiveCount[1]) )
#best 401
print(posWordCount)


In [ ]:
#negative words wordcloud
negWordCount = pd.DataFrame()

negativeWords = []
negativeCount = []
for word in distinctwords:
    if (sid.polarity_scores(word)['compound']) <= -0.5:
        negativeWords.append(word)
        negativeCount.append(totalwords.count(word))


negWordCount['words'] = pd.Series(negativeWords).values
negWordCount['count'] = pd.Series(negativeCount).values



d = {}
for a, x in negWordCount.values:
    d[a] = x

import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud( background_color ='white', max_words=200)
wordcloud.generate_from_frequencies(frequencies=d)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig(abc+'_negative.png', dpi=600) 

plt.show()

#from google.colab import files
#plt.savefig('Output.jpg')
#files.download("Output.jpg")

#print("{0} : {1}".format(positiveWords[1],positiveCount[1]) )
#bad 653
#print(negWordCount.head(10))


In [ ]:
files.download('overall.png')
files.download('positive.png')
files.download('negative.png')

In [ ]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

# Read the whole text.
#text = open(path.join(d, 'alice.txt')).read()
#text = ' '.join(wordcount['words'])
alice_mask = np.array(Image.open(path.join(d, "alice_mask.png")))

stopwords = set(STOPWORDS)

wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,
               stopwords=stopwords, contour_width=3, contour_color='steelblue')

# generate word cloud
#wc.generate(text)
wc.generate_from_frequencies(frequencies=d1)

# store to file
wc.to_file(path.join(d, "Cleartrip.png"))

# show
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
#plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
files.download(abc+'_overall.png')
files.download(abc+'_positive.png')
files.download(abc+'_negative.png')


In [ ]:
print(d)